In [ ]:
import numpy as np
import icechunk
from nimblend import Array, to_zarr, from_zarr, to_icechunk, from_series, to_series

In [ ]:
# Create two arrays with different coordinates
data1 = np.array([[1, 2], [3, 4]])
coords1 = {'x': ['a', 'b'], 'y': [0, 1]}
arr1 = Array(data1, coords1)

data2 = np.array([[10, 20, 30], [40, 50, 60]])
coords2 = {'x': ['b', 'c'], 'y': [0, 1, 2]}
arr2 = Array(data2, coords2)

# Operation automatically aligns coordinates
result = arr1 + arr2
print(result)

# Create a lazy array with Dask
lazy_arr = Array(data1, coords1, chunks="auto")
print(lazy_arr.is_lazy)  # True

# Save array to Zarr format
to_zarr(result, "result.zarr")

# Load from Zarr
loaded = from_zarr("result.zarr")

In [ ]:
storage = icechunk.local_filesystem_storage("/workspaces/nimblend/dataset")
repo = icechunk.Repository.open_or_create(storage)
session = repo.writable_session("main")

In [ ]:
def create_example_arrays():
    # Create some sample data
    data1 = np.array([
        [[1, 2, 3], [4, 5, 6]],
        [[7, 8, 9], [10, 11, 12]],
    ])

    data2 = np.array([
        [[13, 14, 15], [16, 17, 18]],
        [[19, 20, 21], [22, 23, 24]],
    ])

    # Define coordinates with strings for dimensions
    coords1 = {
        'x': np.array(['A', 'B']),
        'y': np.array(['one', 'two']),
        'z': np.array(['i', 'ii', 'iii'])
    }

    coords2 = {
        'x': np.array(['C', 'D']),
        'y': np.array(['three', 'four']),
        'z': np.array(['iv', 'v', 'vi'])
    }

    # Create NimbleNd Arrays
    arr1 = Array(data1, coords=coords1, dims=['x', 'y', 'z'], name='Array 1')
    arr2 = Array(data2, coords=coords2, dims=['x', 'y', 'z'], name='Array 2')

    return arr1, arr2

In [ ]:
arr1, arr2 = create_example_arrays()

In [ ]:
to_icechunk(result, session)

In [ ]:
def test_operations():
    # Create example arrays
    arr1, arr2 = create_example_arrays()

    # Test addition
    print("Adding arrays:")
    result_add = arr1 + arr2
    print(result_add)

    # Export to Zarr format
    print("\nExporting Array 1 to Zarr format...")
    to_zarr(arr1, 'arr1.zarr')

    # Import from Zarr format
    print("\nImporting Array from Zarr format...")
    imported_arr1 = from_zarr('arr1.zarr')
    print(imported_arr1)

    # Example usage of to_icechunk would go here (ensure icechunk is installed)
    # Assuming a session is created and available as `session`
    # print("\nExporting Array 1 to IceChunk format...")
    # to_icechunk(arr1, session, group='example_group')

    # Example series conversion (using strings for Series coordinates)
    print("\nConverting Array 1 to a pandas Series:")
    series_arr1 = to_series(arr1, format='pandas')
    print(series_arr1)

    print("\nConverting Series back to NimbleNd Array:")
    converted_arr1 = from_series(series_arr1)
    print(converted_arr1)